In [55]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [133]:
#придумаем датасет о баллах, полученных на экзаменах в американской школе (Оклахома-Сити)
#math, lang, sing - баллы по предметам,  daylight_hours - продолжительность светового дня в день экзамена
#school_psy - посещал ли ученик школьного психолога, avto_price - стоимость автомобиля,
#полученного учеником от родителей по окончании школы (целевой параметр)

n_samples = 1000

#np.random.seed(0)

math = np.random.choice(30, n_samples) + 15
lang = np.random.choice(50, n_samples) + 25
sing = np.random.choice(84, n_samples) + 16
daylight_hours = np.random.choice(5, n_samples) / 10 + 14
school_psy = np.random.choice([0, 1], n_samples)

avto_price = (math * lang + sing) * 0.8 + 123.0

df = pd.DataFrame({'math':math, 'lang':lang, 'sing':sing, 'daylight_hours':daylight_hours, \
                   'school_psy': school_psy, 'avto_price':avto_price})

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   math            1000 non-null   int64  
 1   lang            1000 non-null   int64  
 2   sing            1000 non-null   int64  
 3   daylight_hours  1000 non-null   float64
 4   school_psy      1000 non-null   int64  
 5   avto_price      1000 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 47.0 KB


,math,lang,sing,daylight_hours,school_psy,avto_price
0,38,61,16,14.3,1,1990.2
1,38,71,74,14.3,0,2340.6
2,17,34,57,14.2,0,631.0
3,20,26,94,14.3,1,614.2
4,42,28,70,14.3,0,1119.8


In [134]:
#В1 - включим все признаки в модель
lr = LinearRegression()

X = df.drop('avto_price', axis = 1)
Y = df['avto_price']

lr.fit(X,Y)
pred_Y = lr.predict(X)

print(f'Веса модели: {lr.coef_}')
print(f'Смещение: {lr.intercept_}')
print(f'MAE: {mean_absolute_error(pred_Y, Y)}')
print(f'RMSE: {np.sqrt(mean_squared_error(pred_Y, Y))}')

Веса модели: [39.89749044 23.71471896  0.55473085 32.4930048  -2.92908565]
Смещение: -1506.1643344764404
MAE: 76.8544744851543
RMSE: 100.88983249765633


получили параметры модели, большое смещение и не самые большие ошибки;
если рассчитать модель для, например, сэмпла "0", то получим значение 1926.617 (при истинном 1990.2), что
укладывается в диапазон ошибки 76.85

In [135]:
#В2 - включим только оценки в модель
lr = LinearRegression()

X = df[['math', 'lang', 'sing']]
Y = df['avto_price']

lr.fit(X,Y)
pred_Y = lr.predict(X)

print(f'Веса модели: {lr.coef_}')
print(f'Смещение: {lr.intercept_}')
print(f'MAE: {mean_absolute_error(pred_Y, Y)}')
print(f'RMSE: {np.sqrt(mean_squared_error(pred_Y, Y))}')

Веса модели: [39.92630556 23.69944535  0.55576042]
Смещение: -1046.5212600876778
MAE: 76.95308481908624
RMSE: 101.00372296577133


коэффициенты параметров math, lang, sing практически не изменились (вероятно, модель определила устойчивую зависимость от них целевой переменной), смещение уменьшилось по сравнению с В1 (из-за отсутствия влияния параметров daylight_hours и school_psy, вероятно, модель списывает на них всю разницу в истинных и предсказанных значениях после взвешивания math, lang и sing), ошибки практически одинаковы

In [145]:
#В3 - введем синтетическую переменную в датасет, соответствующую переменной, примененной для рассчета
#avto_price
lr = LinearRegression()

df['avg_score'] = df['math'] * df['lang'] + df['sing']

X = df[['avg_score']]
Y = df['avto_price']

lr.fit(X,Y)
pred_Y = lr.predict(X)

print(f'Веса модели: {lr.coef_}')
print(f'Смещение: {lr.intercept_}')
print(f'MAE: {mean_absolute_error(pred_Y, Y)}')
print(f'RMSE: {np.sqrt(mean_squared_error(pred_Y, Y))}')

Веса модели: [0.8]
Смещение: 122.99999999999955
MAE: 1.750208866724279e-13
RMSE: 2.531989902174324e-13


модель определила зависимость, правильно подобрала коэффициент (0.8) и смещение (122.99999), ошибки исчезающе малы;

Вывод: очень важно правильно определить значимые параметры, но лучше бы спрогнозировать вид зависимости Y от X-ов